In [30]:
import pandas as pd
import numpy as np
import os
import re

#set data paths
os.chdir("/Users/quinnmackay/Documents/GitHub/BICC/Holcene Revision")
links_path = "EDML-GICC Errors/NGRIP_EDML_match.xlsx"
GICC_GRIP_age = "GICC05-GICC21 Conversion.xlsx"
EDML_age = "EDML-GICC Errors/EDML_counted.xlsx"

#import data
links = pd.read_excel(links_path, skiprows=11, names=["GICC05 b2k", "EDML (m)", "NGRIP (m)"], usecols=[0,1,2])
links['GICC05 b1950'] = links['GICC05 b2k'] - 50
GICC_GRIP_age = pd.read_excel(GICC_GRIP_age, sheet_name=4, skiprows=7, names=["GICC21 Age b2k", "GICC21 dt", "GICC05 years b2k", "GICC05 offset"])
EDML = pd.read_excel(EDML_age, sheet_name=0, skiprows=1, usecols=[0,1])
EDML["age (b1950)"] = EDML["Year b2k"] - 50

EDML_GICC_compare = links[["EDML (m)"]].copy(deep=True) #create a new dataframe for WDC and GRIP depths using stratigraphic links

#get interp age for WDC depths from layer count
interp_edml_age = np.interp(EDML_GICC_compare["EDML (m)"], EDML["Depth (m)"], EDML["age (b1950)"]) #interpolate WDC depths to get ages

EDML_GICC_compare["EDML age (yr BP1950)"] = interp_edml_age #add interpolated ages to dataframe
EDML_GICC_compare = EDML_GICC_compare[EDML_GICC_compare["EDML age (yr BP1950)"] > 1250]
EDML_GICC_compare = EDML_GICC_compare[EDML_GICC_compare["EDML age (yr BP1950)"] < 3835]

#get interp age for GRIP depths from GICC layer count
links_gicc = np.array(links["GICC05 b2k"])
gicc05_t = np.array(GICC_GRIP_age["GICC05 years b2k"])
gicc21_t = np.array(GICC_GRIP_age["GICC21 Age b2k"])

gicc21_links = np.interp(links_gicc, gicc05_t, gicc21_t)
gicc21_links = gicc21_links[gicc21_links < 3835]
gicc21_links = gicc21_links[gicc21_links > 1250]

EDML_GICC_compare["GICC age (yr BP1950)"] = gicc21_links - 50

In [31]:
EDML_GICC_compare

,EDML (m),EDML age (yr BP1950),GICC age (yr BP1950)
14,117.71,1274.090909,1268.0
15,125.25,1381.781609,1375.0
16,127.65,1416.563636,1409.0
17,143.58,1651.663366,1644.0
18,146.15,1689.396226,1681.0
19,148.27,1719.015625,1709.5
20,152.87,1786.159091,1777.0
21,173.84,2100.276923,2088.0
22,175.32,2124.087500,2113.0
23,192.75,2382.765957,2372.1
